In [1]:
import pandas as pd

# Load German Credit Data directly from the UCI Machine Learning Repository
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data"

# The dataset has no headers, so we define them manually
# This data tells us: Age, Credit History, Purpose, Loan Amount... and did they Default?
columns = [
    'Status_Checking', 'Duration_Month', 'Credit_History', 'Purpose',
    'Credit_Amount', 'Savings', 'Employment_Since', 'Installment_Rate',
    'Status_Sex', 'Guarantors', 'Residence_Since', 'Property', 'Age',
    'Other_Plans', 'Housing', 'Credits_Existing', 'Job', 'Liable_People',
    'Telephone', 'Foreign_Worker', 'Target' # 1 = Good, 2 = Bad (Default)
]

# Separator is a space, not a comma
df_credit = pd.read_csv(url, sep=' ', names=columns)

print("Credit Risk Data Loaded.")
print(f"Rows: {len(df_credit)}")
print("Target Distribution (1=Good, 2=Bad):")
print(df_credit['Target'].value_counts())

Credit Risk Data Loaded.
Rows: 1000
Target Distribution (1=Good, 2=Bad):
Target
1    700
2    300
Name: count, dtype: int64


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

# 1. Clean the Data
# The dataset uses '1' for Good and '2' for Bad. Let's make it standard: 0 = Good, 1 = Bad (Default)
df_credit['Target'] = df_credit['Target'].map({1: 0, 2: 1})

# Convert text columns (e.g., "History: Critical") into numbers (0s and 1s)
# This creates new columns like "Purpose_Car", "Purpose_Education", etc.
df_clean = pd.get_dummies(df_credit, drop_first=True)

# 2. Split the Data
# X = All the data about the person (Age, Loan Amount, Job...)
# y = The Target (Did they default?)
X = df_clean.drop('Target', axis=1)
y = df_clean['Target']

# We hide 20% of the data to test the model later
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Train the Model (Random Forest)
print("Training the Fintech Risk Engine...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

print("Training Complete. The model has learned the patterns of default.")

Training the Fintech Risk Engine...
Training Complete. The model has learned the patterns of default.


In [3]:
# 1. Predict
y_pred = rf_model.predict(X_test)

# 2. Evaluate
print("=== RISK MODEL PERFORMANCE REPORT ===\n")
print(classification_report(y_test, y_pred))

# 3. The Money Metrics (Confusion Matrix)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

print("--- FINANCIAL IMPACT ANALYSIS ---")
print(f"Correctly Approved (True Negatives): {tn} (We made profit)")
print(f"Correctly Rejected (True Positives): {tp} (We saved money)")
print(f"False Alarms (False Positives): {fp} (We rejected a good customer - Opportunity Cost)")
print(f"CATASTROPHIC FAILURES (False Negatives): {fn} (We approved a defaulter - Direct Loss)")

=== RISK MODEL PERFORMANCE REPORT ===

              precision    recall  f1-score   support

           0       0.77      0.91      0.83       141
           1       0.62      0.34      0.44        59

    accuracy                           0.74       200
   macro avg       0.70      0.63      0.64       200
weighted avg       0.73      0.74      0.72       200

--- FINANCIAL IMPACT ANALYSIS ---
Correctly Approved (True Negatives): 129 (We made profit)
Correctly Rejected (True Positives): 20 (We saved money)
False Alarms (False Positives): 12 (We rejected a good customer - Opportunity Cost)
CATASTROPHIC FAILURES (False Negatives): 39 (We approved a defaulter - Direct Loss)


In [4]:
# 1. Retrain with "Balanced" Weights
# This tells the model to penalize mistakes on the "Bad" class more heavily
print("Retraining with Heavy Penalties for Defaults...")
rf_model_weighted = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
rf_model_weighted.fit(X_train, y_train)

# 2. Predict Probabilities (Not just Yes/No)
# Instead of a hard "0 or 1", we get the % risk (e.g., 0.45 chance of default)
y_probs = rf_model_weighted.predict_proba(X_test)[:, 1]

# 3. Move the Goalposts (Threshold Tuning)
# Standard models use 0.50 (50%). We will be stricter.
# If the model thinks there is >30% chance of default, we REJECT (Label it 1).
custom_threshold = 0.30
y_pred_strict = (y_probs > custom_threshold).astype(int)

# 4. The New Scorecard
print("\n=== STRICT RISK MODEL REPORT (Threshold: 30%) ===")
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_strict).ravel()

print(f"Correctly Approved (True Negatives): {tn}")
print(f"Correctly Rejected (True Positives): {tp} (These are the losses we avoided!)")
print(f"False Alarms (False Positives): {fp} (Good customers we annoyed - acceptable loss)")
print(f"CATASTROPHIC FAILURES (False Negatives): {fn} (Did this number go down?)")

# Check the new Recall score
from sklearn.metrics import recall_score
new_recall = recall_score(y_test, y_pred_strict)
print(f"\nNew Recall Score: {new_recall:.2f} (Target: > 0.80)")

Retraining with Heavy Penalties for Defaults...

=== STRICT RISK MODEL REPORT (Threshold: 30%) ===
Correctly Approved (True Negatives): 93
Correctly Rejected (True Positives): 47 (These are the losses we avoided!)
False Alarms (False Positives): 48 (Good customers we annoyed - acceptable loss)
CATASTROPHIC FAILURES (False Negatives): 12 (Did this number go down?)

New Recall Score: 0.80 (Target: > 0.80)
